In [1]:
import tensorflow as tf

In [2]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [4]:
text_vec_layer = tf.keras.layers.TextVectorization(
    split = "character",
    standardize = "lower"
)
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

In [5]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

In [6]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(buffer_size=100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [7]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax"),
])

model.compile(
    loss = "sparse_categorical_crossentropy",
    optimizer = "nadam",
    metrics = ["accuracy"],
)

model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespear_model",
    monitor = "val_accuracy",
    save_best_only = True
)

history = model.fit(
    train_set,
    validation_data = valid_set,
    epochs = 10,
    callbacks = [model_ckpt]
)

Epoch 1/10
  31246/Unknown - 412s 12ms/step - loss: 1.4014 - accuracy: 0.5713

31247/31247 [==============================] - 431s 13ms/step - loss: 1.4014 - accuracy: 0.5713 - val_loss: 1.6040 - val_accuracy: 0.5372
Epoch 2/10
31246/31247 [============================>.] - ETA: 0s - loss: 1.2950 - accuracy: 0.5969

31247/31247 [==============================] - 402s 12ms/step - loss: 1.2950 - accuracy: 0.5969 - val_loss: 1.5796 - val_accuracy: 0.5426
Epoch 3/10
31246/31247 [============================>.] - ETA: 0s - loss: 1.2766 - accuracy: 0.6006

31247/31247 [==============================] - 381s 12ms/step - loss: 1.2766 - accuracy: 0.6006 - val_loss: 1.5697 - val_accuracy: 0.5447
Epoch 4/10
31247/31247 [==============================] - 375s 11ms/step - loss: 1.2667 - accuracy: 0.6028 - val_loss: 1.5608 - val_accuracy: 0.5444
Epoch 5/10
31247/31247 [==============================] - 382s 12ms/step - loss: 1.2600 - accuracy: 0.6042 - val_loss: 1.5719 - val_accuracy: 0.5439
Epoch 6/10
31243/31247 [============================>.] - ETA: 0s - loss: 1.2551 - accuracy: 0.6056

31247/31247 [==============================] - 407s 12ms/step - loss: 1.2551 - accuracy: 0.6056 - val_loss: 1.5628 - val_accuracy: 0.5464
Epoch 7/10
31242/31247 [============================>.] - ETA: 0s - loss: 1.2519 - accuracy: 0.6062

31247/31247 [==============================] - 378s 12ms/step - loss: 1.2518 - accuracy: 0.6062 - val_loss: 1.5514 - val_accuracy: 0.5472
Epoch 8/10
31245/31247 [============================>.] - ETA: 0s - loss: 1.2493 - accuracy: 0.6066

31247/31247 [==============================] - 382s 12ms/step - loss: 1.2493 - accuracy: 0.6066 - val_loss: 1.5489 - val_accuracy: 0.5476
Epoch 9/10
31244/31247 [============================>.] - ETA: 0s - loss: 1.2471 - accuracy: 0.6070

31247/31247 [==============================] - 383s 12ms/step - loss: 1.2471 - accuracy: 0.6070 - val_loss: 1.5486 - val_accuracy: 0.5498
Epoch 10/10
31247/31247 [==============================] - 405s 12ms/step - loss: 1.2453 - accuracy: 0.6074 - val_loss: 1.5514 - val_accuracy: 0.5483


In [9]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),
    model
])

In [10]:
y_proba = shakespeare_model.predict(["To be or not to b"])[0, -1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 1s 555ms/step


'e'

# Generating Fake Shakespeare Text

In [11]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 0, 1, 1, 1, 0, 0, 0]])>

In [12]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [13]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)

    return text

In [14]:
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature=0.01))

1/1 [==============================] - 0s 39ms/step
To be or not to be a princely and the duke
that i should be a provos


In [15]:
print(extend_text("To be or not to be", temperature=1))

1/1 [==============================] - 0s 57ms/step
To be or not to begince
and seal'd your to him rich and thrive
as al


In [16]:
print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 39ms/step
To be or not to bepevicm-vilv!?$ez?gmjz :3?ljb'va;!td&hi.ur3l'-j!3eu
